# 🔍 Embeddings y Similitud en Datos

Objetivo: usar embeddings para búsqueda semántica en catálogos de datos, detección de duplicados, clustering, y recomendaciones.

- Duración: 90 min
- Dificultad: Media
- Stack: OpenAI Embeddings, scikit-learn, FAISS

## 1. Generar embeddings

In [ ]:
import os
import numpy as np
from openai import OpenAI

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def embed(text: str) -> np.ndarray:
    resp = client.embeddings.create(
        model='text-embedding-ada-002',
        input=text
    )
    return np.array(resp.data[0].embedding)

textos = [
    'Tabla de ventas con transacciones diarias',
    'Data de transacciones de venta por día',  # Similar
    'Catálogo de productos con precios',
    'Información demográfica de clientes'
]

embeddings = [embed(t) for t in textos]
print(f'Embeddings generados: {len(embeddings)} vectores de dimensión {len(embeddings[0])}')

## 2. Similitud coseno

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Matriz de similitud
sim_matrix = cosine_similarity(embeddings)

print('Matriz de similitud:\n')
for i, texto in enumerate(textos):
    print(f'{i}. {texto}')

print('\n', sim_matrix.round(3))
print(f'\n➡️ Textos 0 y 1 tienen similitud: {sim_matrix[0][1]:.3f} (duplicados potenciales)')

## 3. Búsqueda semántica en catálogo

In [ ]:
import pandas as pd

# Catálogo de data assets
catalogo = pd.DataFrame([
    {'asset': 'dwh.ventas', 'desc': 'Transacciones de venta con fecha, monto, producto'},
    {'asset': 'dwh.clientes', 'desc': 'Datos demográficos de clientes: edad, ciudad, segmento'},
    {'asset': 'dwh.productos', 'desc': 'Catálogo de productos con precios y categorías'},
    {'asset': 'dwh.inventario', 'desc': 'Stock disponible por almacén y SKU'},
    {'asset': 'analytics.revenue_monthly', 'desc': 'Ingresos mensuales agregados por región'}
])

# Embeddings del catálogo
catalogo['embedding'] = catalogo['desc'].apply(lambda x: embed(x))

def search_catalog(query: str, top_k: int = 3) -> pd.DataFrame:
    query_emb = embed(query)
    catalogo['similarity'] = catalogo['embedding'].apply(
        lambda x: cosine_similarity([query_emb], [x])[0][0]
    )
    return catalogo.nlargest(top_k, 'similarity')[['asset', 'desc', 'similarity']]

print(search_catalog('¿Dónde encuentro información de productos?'))
print('\n')
print(search_catalog('Necesito datos de ventas mensuales'))

## 4. Detección de duplicados

In [ ]:
# Dataset con posibles duplicados
descripciones = [
    'Apple iPhone 13 Pro Max 256GB',
    'iPhone 13 Pro Max 256GB Apple',
    'Samsung Galaxy S21 Ultra',
    'Galaxy S21 Ultra by Samsung',
    'Sony PlayStation 5 Console'
]

embs = [embed(d) for d in descripciones]
threshold = 0.95

duplicates = []
for i in range(len(embs)):
    for j in range(i+1, len(embs)):
        sim = cosine_similarity([embs[i]], [embs[j]])[0][0]
        if sim > threshold:
            duplicates.append((i, j, sim))

print('Duplicados detectados:\n')
for i, j, sim in duplicates:
    print(f'- "{descripciones[i]}" ≈ "{descripciones[j]}" (sim={sim:.3f})')

## 5. Clustering con K-Means

In [ ]:
from sklearn.cluster import KMeans

productos = [
    'Laptop Dell XPS 13',
    'MacBook Pro 14 inch',
    'HP Pavilion Laptop',
    'Nike Air Max Sneakers',
    'Adidas Ultraboost Shoes',
    'Puma Running Shoes',
    'Organic Green Tea 100 bags',
    'Colombian Coffee Beans 1kg'
]

prod_embs = np.array([embed(p) for p in productos])

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(prod_embs)

df_clusters = pd.DataFrame({'producto': productos, 'cluster': clusters})
print(df_clusters.sort_values('cluster'))

## 6. FAISS para búsqueda rápida

In [ ]:
# pip install faiss-cpu
import faiss

# Crear índice
dimension = prod_embs.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(prod_embs.astype('float32'))

# Búsqueda
query = 'zapatos deportivos'
query_emb = embed(query).reshape(1, -1).astype('float32')

k = 3
distances, indices = index.search(query_emb, k)

print(f'Top {k} productos para "{query}":\n')
for i, idx in enumerate(indices[0]):
    print(f'{i+1}. {productos[idx]} (distancia={distances[0][i]:.2f})')

## 7. Recomendaciones

In [ ]:
def recomendar_similar(producto: str, catalogo: list, top_k: int = 3) -> list:
    """Recomienda productos similares."""
    query_emb = embed(producto)
    catalogo_embs = [embed(p) for p in catalogo]
    
    similarities = [
        (p, cosine_similarity([query_emb], [e])[0][0])
        for p, e in zip(catalogo, catalogo_embs)
        if p != producto
    ]
    
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]

comprado = 'Laptop Dell XPS 13'
recomendaciones = recomendar_similar(comprado, productos)

print(f'Cliente compró: {comprado}')
print('Recomendaciones:\n')
for prod, sim in recomendaciones:
    print(f'- {prod} (similitud={sim:.3f})')

## 8. Buenas prácticas

- **Cache embeddings**: generarlos es costoso, almacena en DB.
- **Batch processing**: genera embeddings en lotes para eficiencia.
- **Normalización**: limpia texto antes de generar embeddings.
- **Modelos locales**: considera Sentence Transformers para evitar API calls.
- **Threshold dinámico**: ajusta según caso de uso (duplicados vs búsqueda).
- **Monitoreo de costos**: embeddings consumen tokens.

## 9. Ejercicios

1. Construye un deduplicador de registros usando embeddings.
2. Implementa búsqueda híbrida (keyword + semántica) en tu catálogo.
3. Crea clusters de clientes basados en descripciones de comportamiento.
4. Desarrolla un sistema de recomendación de datasets para analistas.